In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
locale_name = "za"
read_enc_val ="ISO-8859-1"
read_enc_val ="cp1252"
read_enc_val ="utf-8"

In [3]:
global_music_file_paths = glob2.glob('spotify ' + locale_name + ' v2/*.csv')

In [4]:
print(len(global_music_file_paths))

370


In [5]:
print(global_music_file_paths[:4])

['spotify za v2\\regional-za-weekly-2018-03-15.csv', 'spotify za v2\\regional-za-weekly-2018-03-22.csv', 'spotify za v2\\regional-za-weekly-2018-03-29.csv', 'spotify za v2\\regional-za-weekly-2018-04-05.csv']


In [6]:
# global_music_file_paths

#### Add Dates to the Files

In [8]:
#Add the dates to the files - run once
def addsDatesToData():
    to_remove_front = "spotify " +locale_name+ " v2\\regional-" +locale_name+"-weekly-"
    to_remove_back = ".csv"

    len_front_remove = len(to_remove_front)
    len_back_remove = len(to_remove_back)

    for file in global_music_file_paths: 
#         print(file)
        the_data_in_file = pd.read_csv(file)
#         the_data_in_file = the_data_in_file.iloc[1: , :]
#         the_data_in_file.columns = ["Position", "Track Name", "Artist", "Streams", "URL"]
        
        
        the_dates = file[len_front_remove:]
        the_dates = the_dates[:-len_back_remove]

        split_dates = the_dates.split("-")

        start_date = split_dates[0] + "-" + split_dates[1] + "-" + split_dates[2]
        

        len_dates = the_data_in_file.shape[0]
        start_dates = len_dates * [start_date]

        date_columns = {"End Date": start_dates}   

        the_data_in_file["End Date"] = start_dates

        #write the data back to the File
        the_data_in_file.to_csv(file, index=False)
        
#remove dates in existing files
def removeExistingDates():
    for file in global_music_file_paths:
        
        the_data_in_file = pd.read_csv(file)

        the_data_in_file.drop(["Start Date", "End Date"], axis=1, inplace=True) 

        the_data_in_file.to_csv(file, index=False)
    
#check for dates ... opting for removal and putting them back on


In [9]:
# removeExistingDates()
addsDatesToData()

In [10]:
#Combine the files and sort by dates
all_files = []
trouble_files = []
for file in global_music_file_paths:
    the_data_in_file = pd.read_csv(file)
    
    if("URL" in list(the_data_in_file.columns)):
    
        the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
    
    list_vals = sum(list(the_data_in_file.isnull().sum()))
    
    if(list_vals > 0):
        trouble_files.append(file)
    all_files.append(the_data_in_file)
    
the_spotify_data = pd.concat(all_files) 
# trouble_files_df = pd.concat(trouble_files)

In [11]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [12]:
the_spotify_data.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams', 'End Date'],
      dtype='object')

In [13]:
# the_spotify_data

In [14]:
# the_spotify_data["End Date"].unique().tolist()

### Error checking that all the Thursday dates are present

In [16]:
date_list = the_spotify_data["End Date"].unique().tolist()
start_date = date_list[0]
end_date = date_list[len(date_list)-1]

In [17]:
len(date_list)

370

In [18]:
start_date, end_date

('2018-03-15', '2025-04-10')

In [19]:
def check_thursdays_in_range(date_list, start_date, end_date):
    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Initialize a set to store Thursdays in the range
    thursdays_in_range = set()
    
    # Iterate through the range of dates and add Thursdays to the set
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 3:  # Thursday
            thursdays_in_range.add(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    
    # Convert date_list to a set for efficient membership checking
    date_set = set(date_list)
    
    # Find missing Thursdays
    missing_thursdays = sorted(thursdays_in_range - date_set)
    print("thursdays_in_range: ", len(thursdays_in_range))
    print("date_set: ", len(date_set))
    
    if missing_thursdays:
        print("Missing Thursdays in the range:")
        for thursday in missing_thursdays:
            print(thursday)
        return False
    else:
        print("All Thursdays are present in the range.")
        return True

In [20]:
result = check_thursdays_in_range(date_list, start_date, end_date)
print("All Thursdays present:", result)

thursdays_in_range:  370
date_set:  370
All Thursdays are present in the range.
All Thursdays present: True


### Continue

In [22]:
the_spotify_data["Week"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["Year"] = the_spotify_data["Week"]

the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )

# the_spotify_data["Start Date Dt"] = the_spotify_data["Start Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["End Date Dt"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data['Seconds since Epoch'] = the_spotify_data['End Date Dt'].apply(lambda x : round(x.timestamp(), 0))

In [23]:
#get the month when this happened
def changedaytotheFirst(x):
    x = x[:-2] + "01"
    
    return x

In [24]:
the_spotify_data["End Date Aggregated"] = the_spotify_data["End Date"].apply(lambda x : changedaytotheFirst(x) )

In [25]:
#find the number of artists on the track
the_spotify_data.dropna(inplace=True)

the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["artist_names"] = the_spotify_data.apply(lambda x : x["artist_names"] + "Artist_NA" if(x["artist_names"] == "") else x["artist_names"], axis=1)

the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["track_name"] = the_spotify_data.apply(lambda x : x["artist_names"] + "_NA" if(x["track_name"] == "") else x["track_name"], axis=1)

def countNumberArtists(x):
    len_all_featured_artists = 0
    num_artists = 0
    xvals = x.split("feat")
    len_xvals = len(xvals)
    
    if(len_xvals > 1):
        featured_artists = xvals[1:]
        featured_artists_str = ' '.join(featured_artists)
        all_featured_artists = featured_artists_str.split("& ")
        len_all_featured_artists = len(all_featured_artists)
        
    num_artists = 1 + len_all_featured_artists
    
    return num_artists
        
    
the_spotify_data['ArtistCount'] = the_spotify_data['track_name'].apply(lambda x : countNumberArtists(x))

In [26]:
the_spotify_data.shape

(73886, 16)

In [27]:
the_spotify_data["main_artist"] = the_spotify_data.apply(lambda x : x["artist_names"].split(",")[0], axis=1)

In [28]:
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["index"], axis=1, inplace=True)

In [29]:
the_spotify_data.shape

(73886, 17)

### Is Top Ten Status

In [31]:
def everInTopTen(x):
    position_vals = list(x["rank"])
    position_vals = list(set(position_vals))
    
#     print(position_vals)
    
    min_pos = min(position_vals)
    existence_val = 0
    
    if(min_pos <= 10):
        existence_val = 1
    
    x["isTopTen"] = [existence_val] * len(x)
    
    return x
    
the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : everInTopTen(x))

C:\Users\lkhum\AppData\Local\Temp\ipykernel_26636\3443158733.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : everInTopTen(x))


In [32]:
the_spotify_data.head()

rank  \
artist_names                 track_name                    
$, Kanye West, Ty Dolla $ign BACK TO ME      61723    38   
                                             61970    85   
                             BEG FORGIVENESS 61759    74   
                             BURN            61764    79   
                                             62002   117   

                                                                                     uri  \
artist_names                 track_name                                                    
$, Kanye West, Ty Dolla $ign BACK TO ME      61723  spotify:track:1icgLGTpX2fQXKRe4D7w2b   
                                             61970  spotify:track:1icgLGTpX2fQXKRe4D7w2b   
                             BEG FORGIVENESS 61759  spotify:track:4ihV1wv7QmjbkLHaT9lA4s   
                             BURN            61764  spotify:track:04CyMEHliadfQWMUJb1w99   
                                             62002  spotify:track:04CyMEHliadfQWMUJb1w99   

                                                                    artist_names  \
artist_names                 track_name                                            
$, Kanye West, Ty Dolla $ign BACK TO ME      61723  $, Kanye West, Ty Dolla $ign   
                                             61970  $, Kanye West, Ty Dolla $ign   
                             BEG FORGIVENESS 61759  $, Kanye West, Ty Dolla $ign   
                             BURN            61764  $, Kanye West, Ty Dolla $ign   
                                             62002  $, Kanye West, Ty Dolla $ign   

                                                         track_name source  \
artist_names                 track_name                                      
$, Kanye West, Ty Dolla $ign BACK TO ME      61723       BACK TO ME     Ye   
                                             61970       BACK TO ME     Ye   
                             BEG FORGIVENESS 61759  BEG FORGIVENESS     Ye   
                             BURN            61764             BURN     Ye   
                                             62002             BURN     Ye   

                                                    peak_rank  previous_rank  \
artist_names                 track_name                                        
$, Kanye West, Ty Dolla $ign BACK TO ME      61723         38             -1   
                                             61970         38             38   
                             BEG FORGIVENESS 61759         74             -1   
                             BURN            61764         79             -1   
                                             62002         79             79   

                                                    weeks_on_chart  streams  \
artist_names                 track_name                                       
$, Kanye West, Ty Dolla $ign BACK TO ME      61723               1   133735   
                                             61970               2    88457   
                             BEG FORGIVENESS 61759               1    95997   
                             BURN            61764               1    91850   
                                             62002               2    79620   

                                                      End Date  Week  Year  \
artist_names                 track_name                                      
$, Kanye West, Ty Dolla $ign BACK TO ME      61723  2024-02-15     7  2024   
                                             61970  2024-02-22     8  2024   
                             BEG FORGIVENESS 61759  2024-02-15     7  2024   
                             BURN            61764  2024-02-15     7  2024   
                                             62002  2024-02-22     8  2024   

                                                   End Date Dt  \
artist_names                 track_name                          
$, Kanye West, Ty Dolla $ign BACK TO ME      61723  2024

### When did the track reach the Top 10 

### how long does a song last on the charts?

In [35]:
the_spotify_data["isTopTen"].value_counts()

isTopTen
0    49954
1    23932
Name: count, dtype: int64

In [36]:
the_spotify_data.isnull().sum()

rank                   0
uri                    0
artist_names           0
track_name             0
source                 0
peak_rank              0
previous_rank          0
weeks_on_chart         0
streams                0
End Date               0
Week                   0
Year                   0
End Date Dt            0
Seconds since Epoch    0
End Date Aggregated    0
ArtistCount            0
main_artist            0
isTopTen               0
dtype: int64

In [37]:
unique_artists = the_spotify_data["main_artist"].unique().tolist()
num_artists = len(unique_artists)
print(num_artists)

1263


### find new artists in that week

In [39]:
# the_spotify_data.head(12)
def getArtistAppearanceCount(x):

    x = x.sort_values(by='End Date', ascending=True)
    x = x.reset_index()
    # x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = [w.replace('index', 'artistAppearanceCount') for w in x.columns]
    
    x.columns = x_cols
    
    return x        

the_spotify_data = the_spotify_data.drop(['artist_names', 'track_name'], axis=1)
the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))
the_spotify_data.drop(["level_2"], inplace=True, axis=1)

C:\Users\lkhum\AppData\Local\Temp\ipykernel_26636\3015101900.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))


In [40]:
the_spotify_data.head()

artistAppearanceCount                  artist_names       track_name  \
0 0                      0  $, Kanye West, Ty Dolla $ign       BACK TO ME   
  1                      1  $, Kanye West, Ty Dolla $ign  BEG FORGIVENESS   
  2                      2  $, Kanye West, Ty Dolla $ign             BURN   
  3                      3  $, Kanye West, Ty Dolla $ign         VULTURES   
  4                      4  $, Kanye West, Ty Dolla $ign          TALKING   

     rank                                   uri source  peak_rank  \
0 0    38  spotify:track:1icgLGTpX2fQXKRe4D7w2b     Ye         38   
  1    74  spotify:track:4ihV1wv7QmjbkLHaT9lA4s     Ye         74   
  2    79  spotify:track:04CyMEHliadfQWMUJb1w99     Ye         79   
  3    83  spotify:track:3SIRBp4RRQ2AO5H4NO7xfq     Ye         83   
  4    31  spotify:track:1eaqMiiUn2P7MnqJK4XeK0     Ye         31   

     previous_rank  weeks_on_chart  streams    End Date  Week  Year  \
0 0             -1               1   133735  2024-02-15     7  2024   
  1             -1               1    95997  2024-02-15     7  2024   
  2             -1               1    91850  2024-02-15     7  2024   
  3             -1               1    91357  2024-02-15     7  2024   
  4             -1               1   143905  2024-02-15     7  2024   

    End Date Dt  Seconds since Epoch End Date Aggregated  ArtistCount  \
0 0  2024-02-15         1.707955e+09          2024-02-01            1   
  1  2024-02-15         1.707955e+09          2024-02-01            1   
  2  2024-02-15         1.707955e+09          2024-02-01            1   
  3  2024-02-15         1.707955e+09          2024-02-01            1   
  4  2024-02-15         1.707955e+09          2024-02-01            1   

    main_artist  isTopTen  
0 0           $         0  
  1           $         0  
  2           $         0  
  3           $         0  
  4           $         0

#### Track Appearance Count

In [42]:
def getTrackAppearanceCount(x):

    x = x.sort_values(by=['End Date','track_name'], ascending=True)

    x = x.reset_index()
    # x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'trackAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x         
    
the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))
# the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)
# print(the_spotify_data.columns)
# stop

C:\Users\lkhum\AppData\Local\Temp\ipykernel_26636\2047836851.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))


In [43]:
the_spotify_data.head()

trackAppearanceCount  artistAppearanceCount  \
0 0                     0                      0   
  1                     1                     19   
1 0                     0                      1   
2 0                     0                      2   
  1                     1                     18   

                     artist_names       track_name  rank  \
0 0  $, Kanye West, Ty Dolla $ign       BACK TO ME    38   
  1  $, Kanye West, Ty Dolla $ign       BACK TO ME    85   
1 0  $, Kanye West, Ty Dolla $ign  BEG FORGIVENESS    74   
2 0  $, Kanye West, Ty Dolla $ign             BURN    79   
  1  $, Kanye West, Ty Dolla $ign             BURN   117   

                                      uri source  peak_rank  previous_rank  \
0 0  spotify:track:1icgLGTpX2fQXKRe4D7w2b     Ye         38             -1   
  1  spotify:track:1icgLGTpX2fQXKRe4D7w2b     Ye         38             38   
1 0  spotify:track:4ihV1wv7QmjbkLHaT9lA4s     Ye         74             -1   
2 0  spotify:track:04CyMEHliadfQWMUJb1w99     Ye         79             -1   
  1  spotify:track:04CyMEHliadfQWMUJb1w99     Ye         79             79   

     weeks_on_chart  streams    End Date  Week  Year End Date Dt  \
0 0               1   133735  2024-02-15     7  2024  2024-02-15   
  1               2    88457  2024-02-22     8  2024  2024-02-22   
1 0               1    95997  2024-02-15     7  2024  2024-02-15   
2 0               1    91850  2024-02-15     7  2024  2024-02-15   
  1               2    79620  2024-02-22     8  2024  2024-02-22   

     Seconds since Epoch End Date Aggregated  ArtistCount main_artist  \
0 0         1.707955e+09          2024-02-01            1           $   
  1         1.708560e+09          2024-02-01            1           $   
1 0         1.707955e+09          2024-02-01            1           $   
2 0         1.707955e+09          2024-02-01            1           $   
  1         1.708560e+09          2024-02-01            1           $   

     isTopTen  
0 0         0  
  1         0  
1 0         0  
2 0         0  
  1         0

In [44]:
the_spotify_data.to_csv("the_spotify_data_" + locale_name+ ".csv")

In [45]:
the_spotify_data["rank"] = the_spotify_data["rank"].astype(int)
the_spotify_data["streams"] = the_spotify_data["streams"].astype(int)
the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)

the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
# the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)

In [46]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [47]:
the_spotify_data["Artist and Track"] = the_spotify_data["artist_names"] + "; " + the_spotify_data["track_name"]

### Getting the list of Artists and Tracks

In [49]:
import pandas as pd
# import pymongo
# import getlyrics

In [50]:
def trackAppearance(x):
#     x = x.reset_index()

    x = x.sort_values(by=["End Date"])
#     x.set_index(["Position"])

    x["rank"] = x["rank"].astype(int)
#     print(list(x["Position"]))
    
    x["rank difference"] = x["rank"].diff().fillna(0)
    x["rank difference"] = x["rank difference"].astype(int)
    
#     x["Position Difference"] = x["Position"].sub(x["Position"].shift())
#     x.drop(["index"], inplace=True)
    
    return x
    
all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: trackAppearance(x))

C:\Users\lkhum\AppData\Local\Temp\ipykernel_26636\2368203610.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: trackAppearance(x))


In [51]:
all_files_grouped["rank difference"].max()

187

### Position over time

In [53]:
def positionvertime(x):
    posovertime = 0
    #new track
    if(x["rank difference"] == 0 and x["trackAppearanceCount"]==0):
        posovertime = 0
    #track stayed in the same position
    elif(x["rank difference"] == 0 and x["trackAppearanceCount"]!=0):
        posovertime = 50
    #track went up the chart
    elif(x["rank difference"] < 0):
        posovertime = 75
    #track fell down the chart
    else:
        posovertime = 100
    return posovertime

In [54]:
all_files_grouped["Position over Time"] = all_files_grouped.apply(lambda x : positionvertime(x), axis=1)

##### Add a Girl Group and Boy Group

In [56]:
ggdf = pd.read_csv("Classification/ListofGirlGroups.csv", encoding='ANSI')
bgdf = pd.read_csv("Classification/ListofBoyGroups.csv", on_bad_lines='skip', encoding='ANSI')

In [57]:
def IsGG(ggdf, y):
    ggs = [x.lower().strip() for x in ggdf["Artist"]]
    y = y.lower().strip()
    
    if y in ggs:
        return True
    else:
        return False

def IsBG(bgdf, y):
    bgs = [x.lower().strip() for x in bgdf["Artist"]]
    y = y.lower().strip()
    
    if y in bgs:
        return True
    else:
        return False

In [58]:
all_files_grouped["IsGirlGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(ggdf, x))
all_files_grouped["IsBoyGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(bgdf, x))

In [59]:
ggs = all_files_grouped[all_files_grouped["IsGirlGroup"]==True]["main_artist"].unique().tolist()
bgs = all_files_grouped[all_files_grouped["IsBoyGroup"]==True]["main_artist"].unique().tolist()

In [60]:
# all_files_grouped

In [61]:
all_files_grouped = all_files_grouped.droplevel(['main_artist', 'track_name'])

In [62]:
# all_files_grouped.drop(["level_2"], inplace=True, axis=1)

In [63]:
all_files_grouped.to_csv("Classification/all_files_"+locale_name+"v1.csv", index=False)

In [64]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [65]:
# new_all_files_grouped = all_files_grouped.reset_index(drop=True)

In [66]:
def newstuff(x):
#     x = x.reset_index()

    x = x.sort_values(by=['Seconds since Epoch'])
#     x.set_index(["Position"])
    print(x)
    
    return x
    


In [67]:
# all_files_grouped = new_all_files_grouped.groupby(["Artist", "Track Name"]).apply(lambda x: newstuff(x))

In [68]:
# all_files_grouped.min()

In [69]:
# df = pd.DataFrame({'A': 'a a b'.split(), 'B': [1,2,3], 'C': [4,6, 5]})

In [70]:
# new_all_files_grouped.groupby([])

In [71]:
# all_files_grouped.apply(lambda x : print(type(x)) )

In [72]:
# # new_all_files_grouped.h'Track Name', 'Artist'ead(5)
# cols_ = ['Start Date', 'Seconds since Epoch', 'Position']
# new_test_df = pd.DataFrame()

In [73]:
# 604800
def weekonweekposition(x):
    
#     print(x.values)
    
    pd.DataFrame({'email':x.index, 'list':x.values})
    
    
#     return x_
    
#     x_ = pd.DataFrame(x, columns=['Start Date'])
    
#     x_ = x.to_frame()
    
#     print(x_.columns)
    
#     for n in range(0, fin_len):
#         for col in cols_:
#             print(len(x[n]), x[n], col)
        
    
#     x_ = pd.DataFrame(x, columns=cols_)
#     print(x_.shape)
#     print(x.columns)

In [74]:
# all_files_gr = new_all_files_grouped[['Artist', 'Track Name', 'Seconds since Epoch', 'Position']]

In [75]:
# all_files_grouped[cols_].apply( lambda x : weekonweekposition(x) )

In [76]:
# new_test_df

### South African Artists

In [78]:
all_files_grouped.shape

(73886, 25)

In [79]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [80]:
narrowing_df = all_files_grouped[['End Date', "rank difference"]]

In [81]:
narrowing_df.reset_index(inplace=True)

In [82]:
# narrowing_df.drop(["level_2"], axis=1, inplace=True)

In [83]:
narrowing_df = narrowing_df[narrowing_df["rank difference"] == 0]

In [84]:
print("done.")

done.
